# 数字图像处理的基本概念

## 数字图像

一幅图像可以看作一个二元函数$f(x,y)$，其中$x, y$是空域坐标，而$f$则是函数的幅度值。当$x, y, f$等取值都是**有限**且**离散**的变量时，这样的图像称为数字图像。

一幅模拟图像要转换为数字图像，需要经过采样、量化等基本过程。

![](https://s2.loli.net/2023/04/08/Fs8ahkVLSdyw5fB.png)

### 量化

概念：将连续的信号幅值近似为有限多个离散值的方法。

方法：
 1. 将信号的幅值区间划分为**连续**、**相互不重叠**的子区间，
 2. 每个子区间**人为**规定一个唯一值，
 3. 落入该子区间的所有信号幅值都是用人为规定的值表示。

根据子区间划分方式的不同，可以将量化分为均匀量化（图片量化中常用）和非均匀量化（移动通信中常用，如语音的PCM编码）。

量化会导致原始信号幅值和量化后的幅值之间存在差异，这种差异称为量化噪声，量化噪声的强度和子区间的数量负相关。

### 数字图像的表示

假设一幅图像具有$M$行，$N$列。

一共有三种方式表示一幅数字图像：
 - 令$z = f(x,y)$，则可以在三维坐标系中绘制图像信号强度关于平面坐标的关系
 - 将$f$映射为对应的灰度（或者颜色），这就是肉眼看到的图像
 - 将其表示为$M \times N$的矩阵，如$(1)$

$$
f(x, y) = \begin{bmatrix}
    f(0, 0) & f(0, 1) & \dots & f(0, N - 1) \\
    f(1, 0) & f(1, 1) & \dots & f(1, N - 1) \\
    \vdots & \vdots & & \vdots \\
    f(M, 0) & f(M, 1) & \dots & f(M - 1, N - 1)
\end{bmatrix} \tag1
$$

接下来，几乎所有数字图像在空域的处理，都基于这个矩阵表现出的特征：

- 矩阵中的每一个元素对应一个像素
- 矩阵左上角的元素是$f(0, 0)$，这意味着在进行图像处理时，我们总是以左上角为坐标原点
- 图像的$x$轴是垂直向下的，$y$轴是水平向右的
- 图像的中心坐标为$(\lfloor \frac M 2 \rfloor, \lfloor \frac N 2 \rfloor)$ 

下面是图像表示的一个例子：

In [ ]:
from imgtools.utils import ASSETS_DIR
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

fig = plt.figure()

with Image.open(ASSETS_DIR / '00_girl.png') as im:
    imat = np.array(im)

    w, h = imat.shape

    xs = range(w)
    ys = range(h)

    # z = f(x, y)
    X, Y = np.meshgrid(xs, ys)
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    ax.plot_surface(X, Y, imat)
    plt.title('z = f(x, y)')

    # imshow
    plt.subplot(1, 2, 2)
    plt.imshow(imat, cmap='gray')
    plt.title('original')

    plt.show()

### 图像的强度

通常使用$L$表示图像的信号强度，一般满足：

$$
0 \le L \le 2^k - 1, \forall k \in N^+ \tag2
$$

一幅数字图像在内存中占用的比特数可以表示为：

$$
b = M \times N \times k \tag3
$$

若图像具有$2^k$个可能的信号强度等级时，我们一般称之为“$k$-bit图像”。

### 内存中的数字图像

在计算机内存中，内存地址是线性增长的，此时按照坐标遍历像素已经不够方便，需要使用“指针”进行线性便利。假设图像是按列首尾相接进行存储的，此时的线性访问称为列扫描：

![](https://s2.loli.net/2023/04/09/FMsCo4PraZwAiIH.png)

假设“指针”在内存中移动的距离为$\alpha$，那么对于大小确定的图像而言，我们可以迅速确定当前像素的坐标值：

$$
\begin{cases}
    x = \alpha \bmod M \\
    y = (\alpha - x) / M
\end{cases} \tag4
$$

## 色彩模型

这些笔记所面向的数字图像，如无特殊说明，一般是指灰度图像或单色图像。对于彩色图像，其色彩是由不同通道(channel)组合而来，且每个通道均可视为单色图像。

但是对彩色图像使用的色彩模型还是有必要了解一下的。一般而言，图像传感器产生的图像信号主要是RGB色彩模型和YCbCr色彩模型。关于色彩模型的推导，可以参考[颜色空间发展简史](https://zhuanlan.zhihu.com/p/498289926)。

### RGB模型

RGB模型其实是对人眼视觉的建模，生物学上认为人眼中存在三种视觉细胞，分别对红、绿、蓝光最敏感，而人能看到多种色彩，是因为三种细胞感光的综合作用。因此红、绿、蓝也被称为“三原色”。

![image.png](https://s2.loli.net/2023/06/20/jFrh8GdlZKMS35C.png)

### YCbCr模型

不同于RGB模型使用三原色组合出丰富多彩的颜色，YCbCr模型的三通道分别是亮度(Y)，蓝色色差(Cb)和红色色差(Cr)。它起源于YUV模型，一种将亮度和色度分离的模拟色彩模型。

对于YUV模型而言，它的通道取值满足$Y \in [0, 1], U,V \in [-0.5, 0.5]$。将YUV模型进行压缩和偏移后就得到了YCbCr模型，因此在数字图像领域的实际应用中，二者常混用。

YCbCr模型可以细分为两种格式：
- TV range: 主要用于广播电视，其中$Y \in [16, 235]$，$U, V \in [16, 240]$。这个取值范围主要是为了避免数模转换过程中由于吉布斯效应引起的过冲现象。
- full range: 主要用于PC，也称PC range，三通道的取值范围均为$[0, 255]$。

### RGB与YCbCr的转换

对于SDTV/HDTV/PC而言，由于YCbCr色域取值范围的不同，因此具体的转换参数也不同。设$T_1, T_2, T'$分别为RGB转YCbCr的参数矩阵、YCbCr转RGB的参数矩阵、偏移矩阵。

那么，RGB转换到YCbCr的过程可以表示为：

$$
\begin{bmatrix} Y \\ C_b \\ C_r \end{bmatrix}
    = T' + T_1 \times \begin{bmatrix} R \\ G \\ B \end{bmatrix} \tag5
$$

而YCbCr转换到RGB可以表示为：

$$
\begin{bmatrix} R \\ G \\ B \end{bmatrix}
    = T_2 \times (\begin{bmatrix} Y \\ C_b \\ C_r \end{bmatrix} - T') \tag6
$$

各个矩阵的取值不再赘述。

特别要强调的一点是，RGB彩色图像转为灰度图像的过程，就是求解Y通道的过程。

## References

1. Chapter 1,2 of *Digital Image Processing 4th edition*
2. 基于Matlab与FPGA的图像处理教程，第二章，韩彬，林海全，姜宇奇